# pandas数据处理

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame

## 1、删除重复元素

使用duplicated()函数检测重复的行，返回元素为布尔类型的Series对象，每个元素对应一行，如果该行不是第一次出现，则元素为True

In [2]:
def make_df(index, columns):
    df = DataFrame({key: [key + str(i) for i in index] for key in columns})
    df.index = index
    return df

In [3]:
df = make_df([1,2,3,4], list('ABCD'))
df


,A,B,C,D
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


In [4]:
df.loc[1] = df.loc[2]
df

,A,B,C,D
1,A2,B2,C2,D2
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


In [7]:
df.duplicated(subset=['B', 'C', 'D'], keep='last')

1     True
2    False
3    False
4    False
dtype: bool

In [8]:
df[df.duplicated()]

,A,B,C,D
2,A2,B2,C2,D2


In [9]:
np.logical_not(df.duplicated())

1     True
2    False
3     True
4     True
dtype: bool

In [10]:
df[np.logical_not(df.duplicated())]

,A,B,C,D
1,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


In [11]:
df[~df.duplicated()]

,A,B,C,D
1,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


使用drop_duplicates()函数删除重复的行

In [13]:
df.drop_duplicates(keep='last')

,A,B,C,D
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


## 2. 映射

映射的含义：创建一个映射关系列表，把values元素和一个特定的标签或者字符串绑定

需要使用字典：

`map = {
    'label1':'value1',
    'label2':'value2',
    ...
    }
`

包含三种操作：

- replace()函数：替换元素
- 最重要：map()函数：新建一列
- rename()函数：替换索引

### 1) replace()函数：替换元素

使用replace()函数，对values进行替换操作

In [14]:
ddd = DataFrame(data=np.random.randint(0,150, size=(3,3)), index=['张三', '李四', '王五'], columns=['语文', '数学', '英语'])
ddd

,语文,数学,英语
张三,125,72,10
李四,7,138,131
王五,116,102,146


In [18]:
ddd.iloc[0,0] = 10

首先定义一个字典

In [15]:
mapping = {10: 100, 7: 107}

调用.replace()

In [22]:
ddd

,语文,数学,英语
张三,10,72,10
李四,7,138,131
王五,116,102,146


In [23]:
ddd.replace([7,10], method='bfill', limit=1)

,语文,数学,英语
张三,10,72,131
李四,116,138,131
王五,116,102,146


replace还经常用来替换NaN元素

In [24]:
ddd = ddd.astype(np.float64)
ddd.iloc[0,0] = np.nan
ddd

,语文,数学,英语
张三,NaN,72.0,10.0
李四,7.0,138.0,131.0
王五,116.0,102.0,146.0


In [25]:
ddd.replace({np.nan: 0.0})

,语文,数学,英语
张三,0.0,72.0,10.0
李四,7.0,138.0,131.0
王五,116.0,102.0,146.0


============================================

练习19：

    假设张三李四的课表里有满分的情况，老师认为是作弊，把所有满分的情况（包括150,300分）都记0分，如何实现？

============================================

In [28]:
index = ['语文', '数学', '英语', '文综']
columns = ['张三', '李四', '王五']
data = np.random.randint(0,150, size=(4,3))
df = DataFrame(data=data, index=index,  columns=columns)
df

,张三,李四,王五
语文,143,22,136
数学,46,45,10
英语,7,113,137
文综,17,136,38


In [29]:
df.iloc[0,0] = 150
df.iloc[1,0] = 150

df.iloc[3, 0] = 300

In [31]:
df.iloc[0,2] = 150

In [32]:
df

,张三,李四,王五
语文,150,22,150
数学,150,45,10
英语,7,113,137
文综,300,136,38


In [34]:
dic = {150: 0, 300: 0}
df.replace(dic, inplace=True)

In [35]:
df

,张三,李四,王五
语文,0,22,0
数学,0,45,10
英语,7,113,137
文综,0,136,38


### 2) map()函数：新建一列

使用map()函数，由已有的列生成一个新列

适合处理某一单独的列。

In [36]:
df

,张三,李四,王五
语文,0,22,0
数学,0,45,10
英语,7,113,137
文综,0,136,38


仍然是新建一个字典

In [37]:
mapping = {22: 122, 45: 145, 113: 131, 136: 106}
# 用已有的列的数据生成新的一列的数据
df['赵六'] = df['李四'].map(mapping)

In [38]:
df

,张三,李四,王五,赵六
语文,0,22,0,122
数学,0,45,10,145
英语,7,113,137,131
文综,0,136,38,106


map()函数中可以使用lambda函数

In [39]:
df['孙八'] = df['李四'].map(lambda x: x * 2)
df

,张三,李四,王五,赵六,孙八
语文,0,22,0,122,44
数学,0,45,10,145,90
英语,7,113,137,131,226
文综,0,136,38,106,272


transform()和map()类似

In [40]:
df['田七'] = df['李四'].transform(lambda x: x * 2)

In [41]:
df

,张三,李四,王五,赵六,孙八,田七
语文,0,22,0,122,44,44
数学,0,45,10,145,90,90
英语,7,113,137,131,226,226
文综,0,136,38,106,272,272


使用map()函数新建一个新列

In [63]:
columns = ['语文', '数学', '英语']
index = ['张三', '李四', '王五']
data = np.random.randint(0,150, size=(3,3))
df = DataFrame(data=data, index=index,  columns=columns)
df

,语文,数学,英语
张三,123,128,82
李四,109,125,69
王五,139,38,25


In [43]:
def convert(item):
    if item >= 120:
        return '优秀'
    elif item >= 90:
        return '良好'
    else:
        return  '不及格'

In [44]:
df['score_math'] = df['数学'].map(convert)

In [45]:
df

,语文,数学,英语,score_math
张三,55,133,91,优秀
李四,102,58,46,不及格
王五,60,110,37,良好


In [46]:
# 修改已有的数据
df['英语'] = df['英语'].map(lambda x: x+ 20)
df

,语文,数学,英语,score_math
张三,55,133,111,优秀
李四,102,58,66,不及格
王五,60,110,57,良好


============================================

练习20：

    新增两列，分别为张三、李四的成绩状态，如果分数低于90，则为"failed"，如果分数高于120，则为"excellent"，其他则为"pass"
    
    【提示】使用函数作为map的参数

============================================

### 3) rename()函数：替换索引

仍然是新建一个字典

In [47]:
df

,语文,数学,英语,score_math
张三,55,133,111,优秀
李四,102,58,66,不及格
王五,60,110,57,良好


In [48]:
mapping = {'语文': 'Chinese', '数学': 'Math', '英语': 'English'}
df.rename(mapping, axis=1)

,Chinese,Math,English,score_math
张三,55,133,111,优秀
李四,102,58,66,不及格
王五,60,110,57,良好


In [49]:
df.rename(columns=mapping)

,Chinese,Math,English,score_math
张三,55,133,111,优秀
李四,102,58,66,不及格
王五,60,110,57,良好


In [50]:
mapping = {'张三': 'Zhang Sir', '李四': 'Li Sir', '王五': 'LaoWang'}
df.rename(index=mapping)

,语文,数学,英语,score_math
Zhang Sir,55,133,111,优秀
Li Sir,102,58,66,不及格
LaoWang,60,110,57,良好


使用rename()函数替换行索引

## 3. 异常值检测和过滤

使用describe()函数查看每一列的描述性统计量

使用std()函数可以求得DataFrame对象每一列的标准差

根据每一列的标准差，对DataFrame元素进行过滤。

借助any()函数, 测试是否有True，有一个或以上返回True，反之返回False

对每一列应用筛选条件,去除标准差太大的数据

删除特定索引df.drop(labels,inplace = True)

============================================

练习21：

    新建一个形状为10000*3的标准正态分布的DataFrame(np.random.randn)，去除掉所有满足以下情况的行：其中任一元素绝对值大于3倍标准差

============================================

In [51]:
df = DataFrame(data=np.random.randn(10000, 3))
df.head()

,0,1,2
0,0.061293,1.566377,-0.347567
1,0.683608,0.006871,0.146835
2,-0.009825,-1.632561,-1.222459
3,0.076959,-0.554477,0.374802
4,0.425893,1.657765,-0.684815


In [52]:
df.std()

0    0.997193
1    1.010044
2    0.996605
dtype: float64

In [53]:
df.describe()

,0,1,2
count,10000.000000,10000.000000,10000.000000
mean,-0.009125,-0.012397,-0.011071
std,0.997193,1.010044,0.996605
min,-4.023315,-3.649615,-3.716140
25%,-0.676084,-0.698411,-0.687240
50%,-0.000593,-0.012361,-0.031456
75%,0.664914,0.668303,0.651280
max,3.969733,4.116441,3.831404


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
0    10000 non-null float64
1    10000 non-null float64
2    10000 non-null float64
dtypes: float64(3)
memory usage: 234.5 KB


In [60]:
# 判断异常数据.
# 大于三倍标准偏差
cond = (df.abs() > df.std() * 3).any(axis=1)
cond

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20       True
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
9970    False
9971    False
9972    False
9973    False
9974    False
9975    False
9976    False
9977    False
9978    False
9979    False
9980    False
9981    False
9982    False
9983    False
9984    False
9985    False
9986    False
9987    False
9988    False
9989    False
9990    False
9991    False
9992    False
9993    False
9994    False
9995    False
9996    False
9997    False
9998    False
9999    False
Length: 10000, dtype: bool

In [62]:
df[~cond]

,0,1,2
0,0.061293,1.566377,-0.347567
1,0.683608,0.006871,0.146835
2,-0.009825,-1.632561,-1.222459
3,0.076959,-0.554477,0.374802
4,0.425893,1.657765,-0.684815
5,-1.316408,0.140757,-0.561564
6,-0.740842,-0.521429,-2.593181
7,0.525563,-0.577119,0.512401
8,1.764777,-1.823578,-0.456538
9,0.487698,0.374920,0.846084


## 4. 排序

使用.take()函数排序

可以借助np.random.permutation()函数随机排序

In [64]:

df

,语文,数学,英语
张三,123,128,82
李四,109,125,69
王五,139,38,25


In [65]:
# 按照我们指定的 顺序去排序
df.take([2,0,1])

,语文,数学,英语
王五,139,38,25
张三,123,128,82
李四,109,125,69


In [66]:
df.take([2,1,0], axis=1)

,英语,数学,语文
张三,82,128,123
李四,69,125,109
王五,25,38,139


### 随机抽样

In [85]:
np.random.permutation(np.arange(0,5))

array([1, 4, 0, 2, 3])

In [ ]:
随机抽样有两种,一种叫做有放回的随机抽样.
另一种叫做无放回的随机抽样.

In [100]:
# 无放回抽样
df.take(np.random.permutation([0,1,2]))

,语文,数学,英语
张三,123,128,82
王五,139,38,25
李四,109,125,69


当DataFrame规模足够大时，直接使用np.random.randint()函数，就配合take()函数实现随机抽样

In [108]:
np.random.randint(0,10, size=10)

array([8, 7, 8, 1, 6, 7, 1, 1, 4, 7])

In [111]:
df.take(np.random.randint(0,3,size=3))

,语文,数学,英语
李四,109,125,69
李四,109,125,69
李四,109,125,69


============================================
练习22：

   假设有张三李四王老五赵小六的期中考试成绩ddd2，对着三名同学随机排序

============================================

In [112]:
columns = ['语文', '数学', '英语']
index = ['张三', '李四', '王五', '赵小六']
data = np.random.randint(0,150, size=(4,3))
df = DataFrame(data=data, index=index,  columns=columns)
df

,语文,数学,英语
张三,52,145,64
李四,125,5,28
王五,58,110,136
赵小六,104,2,33


In [119]:
df.take(np.random.permutation([0,1,2,3])[:3])

,语文,数学,英语
赵小六,104,2,33
张三,52,145,64
王五,58,110,136


## 5. 数据聚合【重点】

数据聚合是数据处理的最后一步，通常是要使每一个数组生成一个单一的数值。

数据分类处理：

 - 分组：先把数据分为几组
 - 用函数处理：为不同组的数据应用不同的函数以转换数据
 - 合并：把不同组得到的结果合并起来
 
数据分类处理的核心：
     groupby()函数

In [ ]:
数据聚合的套路
1, 分组, groupby
2, 调用聚合 函数,
3, 把聚合的结果追加到原始数据表.

如果想使用color列索引，计算price1的均值，可以先获取到price1列，然后再调用groupby函数，用参数指定color这一列

In [120]:
df = DataFrame({'color':['red','white','red','cyan','cyan','green','white','cyan'],
                'price':np.random.randint(0,8,size = 8),
                'weight':np.random.randint(50,55,size = 8)})
df

,color,price,weight
0,red,1,51
1,white,7,52
2,red,1,53
3,cyan,6,52
4,cyan,4,52
5,green,6,54
6,white,7,53
7,cyan,7,54


使用.groups属性查看各行的分组情况：

In [122]:
# 使用groupby进行 分组
df.groupby(by='color')

In [123]:
df.groupby(by='color').groups

{'cyan': Int64Index([3, 4, 7], dtype='int64'),
 'green': Int64Index([5], dtype='int64'),
 'red': Int64Index([0, 2], dtype='int64'),
 'white': Int64Index([1, 6], dtype='int64')}

In [125]:
# 调用聚合函数
df.groupby(by='color').sum()[['price']] # 会计算所有列

,price
color,
cyan,17
green,6
red,2
white,14


In [129]:
price_sum = df.groupby(by='color')[['price']].sum() # 先取列再聚合. 推荐写法.

In [128]:
df

,color,price,weight
0,red,1,51
1,white,7,52
2,red,1,53
3,cyan,6,52
4,cyan,4,52
5,green,6,54
6,white,7,53
7,cyan,7,54


In [131]:
pd.merge(df, price_sum, left_on='color', right_index=True, suffixes=['', '_sum'])

,color,price,weight,price_sum
0,red,1,51,2
2,red,1,53,2
1,white,7,52,14
6,white,7,53,14
3,cyan,6,52,17
4,cyan,4,52,17
7,cyan,7,54,17
5,green,6,54,6


============================================

练习23：

   假设菜市场张大妈在卖菜，有以下属性：
   
   菜品(item)：萝卜，白菜，辣椒，冬瓜
   
   颜色(color)：白，青，红
   
   重量(weight)
   
   价格(price)
   
1. 要求以属性作为列索引，新建一个ddd
2. 对ddd进行聚合操作，求出颜色为白色的价格总和
3. 对ddd进行聚合操作，求出萝卜的所有重量(包括白萝卜，胡萝卜，青萝卜）以及平均价格
4. 使用merge合并总重量及平均价格

============================================

In [132]:
ddd = DataFrame({'item': ['萝卜','萝卜','萝卜', '白菜', '白菜', '辣椒', '辣椒', '冬瓜', '冬瓜'], 'color': ['白', '青', '红', '白', '青',  '青', '红', '青', '白'], 
                 'weight': [60, 50, 70, 80, 40, 100, 90, 120, 110], 'price': [2.98, 3.98, 5.98, 4.68, 1.98, 9.68, 12.88, 1.28, 3.58]})
ddd

,item,color,weight,price
0,萝卜,白,60,2.98
1,萝卜,青,50,3.98
2,萝卜,红,70,5.98
3,白菜,白,80,4.68
4,白菜,青,40,1.98
5,辣椒,青,100,9.68
6,辣椒,红,90,12.88
7,冬瓜,青,120,1.28
8,冬瓜,白,110,3.58


In [133]:
ddd.groupby(by='color')[['price']].sum()

,price
color,
白,11.24
红,18.86
青,16.92


In [134]:
# 萝卜的所有重量
weight_sum = ddd.groupby(by='item')[['weight']].sum()
weight_sum

,weight
item,
冬瓜,230
白菜,120
萝卜,180
辣椒,190


In [135]:
price_mean = ddd.groupby(by='item')[['price']].mean()
price_mean

,price
item,
冬瓜,2.430000
白菜,3.330000
萝卜,4.313333
辣椒,11.280000


In [137]:
ddd = pd.merge(ddd, weight_sum, left_on='item', right_index=True, suffixes=['', '_sum'])

In [138]:
ddd.merge(price_mean, left_on='item', right_index=True, suffixes=['', '_mean'])

,item,color,weight,price,weight_sum,price_mean
0,萝卜,白,60,2.98,180,4.313333
1,萝卜,青,50,3.98,180,4.313333
2,萝卜,红,70,5.98,180,4.313333
3,白菜,白,80,4.68,120,3.330000
4,白菜,青,40,1.98,120,3.330000
5,辣椒,青,100,9.68,190,11.280000
6,辣椒,红,90,12.88,190,11.280000
7,冬瓜,青,120,1.28,230,2.430000
8,冬瓜,白,110,3.58,230,2.430000


## 6.0 高级数据聚合

可以使用pd.merge()函数将聚合操作的计算结果添加到df的每一行  
使用groupby分组后调用加和等函数进行运算，让后最后可以调用add_prefix()，来修改列名

In [140]:
ddd.groupby(by='item')[['price']].mean().add_prefix('mean_')

,mean_price
item,
冬瓜,2.430000
白菜,3.330000
萝卜,4.313333
辣椒,11.280000


In [141]:
ddd.groupby(by='item')[['price']].mean().add_suffix('_mean')

,price_mean
item,
冬瓜,2.430000
白菜,3.330000
萝卜,4.313333
辣椒,11.280000


### 可以使用transform和apply实现相同功能

在transform或者apply中传入函数即可

In [142]:
ddd.groupby(by='item')[['weight']].transform(sum)

,weight
0,180
1,180
2,180
3,120
4,120
5,190
6,190
7,230
8,230


In [143]:
ddd.groupby(by='item')[['weight']].apply(sum)

,weight
item,
冬瓜,230
白菜,120
萝卜,180
辣椒,190


In [144]:
ddd.groupby(by='item').apply(sum)

,item,color,weight,price,weight_sum
item,,,,,
冬瓜,冬瓜冬瓜,青白,230,4.86,460
白菜,白菜白菜,白青,120,6.66,240
萝卜,萝卜萝卜萝卜,白青红,180,12.94,540
辣椒,辣椒辣椒,青红,190,22.56,380


In [145]:
def convert(x):
    return x.sum()

In [146]:
ddd.groupby(by='item').apply(convert)

,item,color,weight,price,weight_sum
item,,,,,
冬瓜,冬瓜冬瓜,青白,230,4.86,460
白菜,白菜白菜,白青,120,6.66,240
萝卜,萝卜萝卜萝卜,白青红,180,12.94,540
辣椒,辣椒辣椒,青红,190,22.56,380


============================================

练习24：

   使用transform与apply实现练习23的功能

============================================